## Libraries

In [50]:
import pandas as pd
import numpy as np
from crepes import ConformalClassifier, ConformalPredictiveSystem
from crepes.extras import hinge, margin, binning, DifficultyEstimator

## Data

### Calibration 

In [51]:
data_dir = "calibration_and_test.xlsx" #directory
df_cal=pd.read_excel(data_dir,sheet_name='conf_score_25') #cal dataset with conf=0.25
df_cal_tp = df_cal[df_cal['TP']==1] # True Positive Predictions Only

# 43 samples from each class
sample_size = 43
df_sampled = df_cal_tp.groupby('pred_obj_id').apply(lambda x: x.sample(sample_size))
df_sampled = df_sampled.reset_index(drop=True)

df_test=pd.read_excel(data_dir,sheet_name='test_conf_score_25') #test dataset with conf=0.25
df_test_filtered = df_test[df_test['pred_obj_id'] != '-']
df_calibration=df_sampled
df_testing=df_test_filtered

In [52]:
from my_functions import (softmax, 
                          softmax_df, 
                          calculate_logit, 
                          calculate_sigmoid, 
                          calculate_softmax,
                          calculate_hinge_scores,
                          calculate_p_values,
                          calculate_prediction_sets)

## Calibration stage

In [53]:
alphas_cal = hinge(
    df_calibration[[
        'ascaris_conf_score',
        'trichuris_conf_score',
        'hookworm_conf_score',
        'schistosoma_conf_score']].values,
    [0,1,2,3],df_calibration[['pred_obj_id']].values
)
cc_std = ConformalClassifier()
cc_std.fit(alphas_cal)
display(cc_std)

ConformalClassifier(fitted=True, mondrian=False)

## Testing stage

### Applying the hinge on the testing dataset

In [54]:
conf_score_columns = [
    'ascaris_conf_score',
    'trichuris_conf_score',
    'hookworm_conf_score',
    'schistosoma_conf_score']
hinge_score_cols = [
    'ascaris_hinge_score', 
    'trichuris_hinge_score', 
    'hookworm_hinge_score', 
    'schistosoma_hinge_score']
df_testing = calculate_hinge_scores(
    df_testing, 
    conf_score_columns, 
    hinge_score_cols)

C:\Users\M\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [55]:
p_value_cols = [
    'ascaris_p_value', 
    'trichuris_p_value', 
    'hookworm_p_value', 
    'schistosoma_p_value']
df_testing = calculate_p_values(
    df_testing, 
    hinge_score_cols, 
    p_value_cols, 
    cc_std)

C:\Users\M\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [56]:
p_set_cols = [
    'ascaris_p_set', 
    'trichuris_p_set', 
    'hookworm_p_set', 
    'schistosoma_p_set']
df_testing = calculate_prediction_sets(
    df_testing, 
    hinge_score_cols, 
    p_set_cols, 
    cc_std,
    confidence=0.99)

In [11]:
df_testing

,filename,image_width,image_height,gt_obj_id,gt_x_center,gt_y_center,gt_obj_width,gt_obj_height,pred_obj_id,pred_x_center,...,hookworm_hinge_score,schistosoma_hinge_score,ascaris_p_value,trichuris_p_value,hookworm_p_value,schistosoma_p_value,ascaris_p_set,trichuris_p_set,hookworm_p_set,schistosoma_p_set
0,11184288_1608_36_af7b24d7-fd97-42a0-acc9-1b7a9...,1280,720,3,0.9605,0.6972,0.0773,0.1583,3,0.961328,...,0.997449,0.395887,0.001714,0.005184,0.000299,0.203312,0,0,0,1
1,11184288_1608_36_af7b24d7-fd97-42a0-acc9-1b7a9...,1280,720,0,0.4059,0.4347,0.0586,0.1278,2,0.406641,...,0.613458,0.990231,0.000101,0.003684,0.060861,0.000211,0,0,0,0
2,11184288_1608_36_af7b24d7-fd97-42a0-acc9-1b7a9...,1280,720,-,-,-,-,-,1,0.405078,...,0.753833,0.984610,0.000483,0.157588,0.003606,0.001686,0,1,0,0
3,11184394_1608_36_af7b24d7-fd97-42a0-acc9-1b7a9...,1280,720,3,0.1703,0.7111,0.0984,0.125,3,0.16875,...,0.996007,0.278384,0.001218,0.004075,0.004814,0.402918,0,0,0,1
4,11184394_1608_36_af7b24d7-fd97-42a0-acc9-1b7a9...,1280,720,-,-,-,-,-,0,0.692187,...,0.997687,0.999338,0.446970,0.004608,0.001257,0.001061,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11869,9354474_1379_31_75602595-167c-4758-8f96-e8ee5e...,1280,720,0,0.6988,0.5833,0.0602,0.0833,0,0.698828,...,0.996884,0.999090,0.815395,0.000394,0.004256,0.005084,1,0,0,0
11870,9354474_1379_31_75602595-167c-4758-8f96-e8ee5e...,1280,720,0,0.4684,0.2347,0.0586,0.1028,0,0.46875,...,0.993931,0.998322,0.975240,0.003041,0.001416,0.003941,1,0,0,0
11871,9357486_1384_31_941af06b-54a3-4552-b75f-4e6e72...,1280,720,0,0.0187,0.7007,0.0375,0.0958,0,0.019141,...,0.993028,0.993571,0.676322,0.004463,0.002231,0.004136,1,0,0,0
11873,9419118_1393_31_55d950b3-e8d5-48b7-b7a7-30014f...,1280,720,-,-,-,-,-,3,0.970312,...,0.911459,0.584649,0.004964,0.000664,0.001582,0.072878,0,0,0,0


In [29]:
# Create a boolean mask for each condition
mask1 = (df_testing['gt_obj_id'] == any) & (df_testing['pred_obj_id'] == any)
mask2 = df_testing['ascaris_p_set'] == 0
mask3 = (df_testing['trichuris_p_set'] == 0) & (df_testing['hookworm_p_set'] == 0) & (df_testing['schistosoma_p_set'] == 0)

# Combine the masks
mask = mask1 & mask2 & mask3

# Filter the DataFrame
df_filtered = df_testing[mask]

# Count the number of rows
count = len(df_filtered)

print(count)

# Create a boolean mask for each condition
mask1 = (df_testing['gt_obj_id'] == 1) & (df_testing['pred_obj_id'] != 1)
mask2 = df_testing['trichuris_p_set'] == 0
mask3 = (df_testing['ascaris_p_set'] == 0) & (df_testing['hookworm_p_set'] == 0) & (df_testing['schistosoma_p_set'] == 0)

# Combine the masks
mask = mask1 & mask2 & mask3

# Filter the DataFrame
df_filtered = df_testing[mask]

# Count the number of rows
count = len(df_filtered)

print(count)

# Create a boolean mask for each condition
mask1 = (df_testing['gt_obj_id'] == 2) & (df_testing['pred_obj_id'] != 2)
mask2 = df_testing['hookworm_p_set'] == 0
mask3 = (df_testing['ascaris_p_set'] == 0) & (df_testing['trichuris_p_set'] == 0) & (df_testing['schistosoma_p_set'] == 0)

# Combine the masks
mask = mask1 & mask2 & mask3

# Filter the DataFrame
df_filtered = df_testing[mask]

# Count the number of rows
count = len(df_filtered)

print(count)

# Create a boolean mask for each condition
mask1 = (df_testing['gt_obj_id'] == 3) & (df_testing['pred_obj_id'] != 3)
mask2 = df_testing['schistosoma_p_set'] == 0
mask3 = (df_testing['ascaris_p_set'] == 0) & (df_testing['trichuris_p_set'] == 0) & (df_testing['hookworm_p_set'] == 0)

# Combine the masks
mask = mask1 & mask2 & mask3

# Filter the DataFrame
df_filtered = df_testing[mask]

# Count the number of rows
count = len(df_filtered)

print(count)

0
19
7
2


### One Class Per Set with TP

In [57]:
def count_rows(df, obj_id, p_set_one, p_set_zero):
    mask1 = (df['gt_obj_id'] == obj_id) & (df['pred_obj_id'] == obj_id)
    mask2 = df[p_set_one] == 1
    mask3 = df[p_set_zero].apply(lambda x: all(val == 0 for val in x), axis=1)
    mask = mask1 & mask2 & mask3
    df_filtered = df[mask]
    count = len(df_filtered)
    return count

p_set_zero = ['ascaris_p_set', 'trichuris_p_set', 'hookworm_p_set', 'schistosoma_p_set']

print(count_rows(df_testing, 0, 'ascaris_p_set', [col for col in p_set_zero if col != 'ascaris_p_set']))
print(count_rows(df_testing, 1, 'trichuris_p_set', [col for col in p_set_zero if col != 'trichuris_p_set']))
print(count_rows(df_testing, 2, 'hookworm_p_set', [col for col in p_set_zero if col != 'hookworm_p_set']))
print(count_rows(df_testing, 3, 'schistosoma_p_set', [col for col in p_set_zero if col != 'schistosoma_p_set']))

8846
454
16
129


### One Class Per Set with FP

In [59]:
def count_rows(df, obj_id, p_set_one, p_set_zero):
    mask1 = (df['gt_obj_id'] != obj_id) & (df['pred_obj_id'] == obj_id)
    mask2 = df[p_set_one] == 1
    mask3 = df[p_set_zero].apply(lambda x: all(val == 0 for val in x), axis=1)
    mask = mask1 & mask2 & mask3
    df_filtered = df[mask]
    count = len(df_filtered)
    return count

p_set_zero = ['ascaris_p_set', 'trichuris_p_set', 'hookworm_p_set', 'schistosoma_p_set']

print(count_rows(df_testing, 0, 'ascaris_p_set', [col for col in p_set_zero if col != 'ascaris_p_set']))
print(count_rows(df_testing, 1, 'trichuris_p_set', [col for col in p_set_zero if col != 'trichuris_p_set']))
print(count_rows(df_testing, 2, 'hookworm_p_set', [col for col in p_set_zero if col != 'hookworm_p_set']))
print(count_rows(df_testing, 3, 'schistosoma_p_set', [col for col in p_set_zero if col != 'schistosoma_p_set']))

1002
83
80
246


In [60]:
# Create a boolean mask for each condition
mask1 = (df_testing['ascaris_p_set'] == 0)
mask2 = (df_testing['trichuris_p_set'] == 0)
mask3 = (df_testing['hookworm_p_set'] == 0)
mask4 = (df_testing['schistosoma_p_set'] == 0)

# Combine the masks
mask = mask1 & mask2 & mask3 & mask4

# Filter the DataFrame
df_filtered = df_testing[mask]

# Count the number of rows
count = len(df_filtered)

print(count)

205


In [37]:
# Create a boolean mask for each condition
mask1 = df_testing['gt_obj_id'].isin(['-', '0', '1', '2', '3'])
mask2 = (df_testing['pred_obj_id'] == 0)
mask3 = (df_testing['ascaris_p_set'] == 0)
mask4 = (df_testing['trichuris_p_set'] == 0)
mask5 = (df_testing['hookworm_p_set'] == 0)
mask6 = (df_testing['schistosoma_p_set'] == 0)

# Combine the masks
mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6

# Filter the DataFrame
df_filtered = df_testing[mask]

# Count the number of rows
count = len(df_filtered)

print(count)

# Create a boolean mask for each condition
mask1 = df_testing['gt_obj_id'].isin(['-', '0', '1', '2', '3'])
mask2 = (df_testing['pred_obj_id'] == 1)
mask3 = (df_testing['ascaris_p_set'] == 0)
mask4 = (df_testing['trichuris_p_set'] == 0)
mask5 = (df_testing['hookworm_p_set'] == 0)
mask6 = (df_testing['schistosoma_p_set'] == 0)

# Combine the masks
mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6

# Filter the DataFrame
df_filtered = df_testing[mask]

# Count the number of rows
count = len(df_filtered)

print(count)

# Create a boolean mask for each condition
mask1 = df_testing['gt_obj_id'].isin(['-', '0', '1', '2', '3'])
mask2 = (df_testing['pred_obj_id'] == 2)
mask3 = (df_testing['ascaris_p_set'] == 0)
mask4 = (df_testing['trichuris_p_set'] == 0)
mask5 = (df_testing['hookworm_p_set'] == 0)
mask6 = (df_testing['schistosoma_p_set'] == 0)

# Combine the masks
mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6

# Filter the DataFrame
df_filtered = df_testing[mask]

# Count the number of rows
count = len(df_filtered)

print(count)

# Create a boolean mask for each condition
mask1 = df_testing['gt_obj_id'].isin(['-', '0', '1', '2', '3'])
mask2 = (df_testing['pred_obj_id'] == 3)
mask3 = (df_testing['ascaris_p_set'] == 0)
mask4 = (df_testing['trichuris_p_set'] == 0)
mask5 = (df_testing['hookworm_p_set'] == 0)
mask6 = (df_testing['schistosoma_p_set'] == 0)

# Combine the masks
mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6

# Filter the DataFrame
df_filtered = df_testing[mask]

# Count the number of rows
count = len(df_filtered)

print(count)

492
57
40
181


In [61]:
def count_rows(df, pred_obj_id):
    mask1 = df['gt_obj_id'].isin(['-', '0', '1', '2', '3'])
    mask2 = (df['pred_obj_id'] == pred_obj_id)
    mask3 = (df['ascaris_p_set'] == 0)
    mask4 = (df['trichuris_p_set'] == 0)
    mask5 = (df['hookworm_p_set'] == 0)
    mask6 = (df['schistosoma_p_set'] == 0)
    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
    df_filtered = df[mask]
    count = len(df_filtered)
    return count

print(count_rows(df_testing, 0))
print(count_rows(df_testing, 1))
print(count_rows(df_testing, 2))
print(count_rows(df_testing, 3))

102
10
7
32


In [43]:
def count_rows(df):
    mask1 = (df['gt_obj_id'] == '-')
    mask2 = (df['pred_obj_id'] == 0)
    mask3 = (df['ascaris_p_set'] == 0)
    mask4 = (df['trichuris_p_set'] == 0)
    mask5 = (df['hookworm_p_set'] == 0)
    mask6 = (df['schistosoma_p_set'] == 0)
    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
    df_filtered = df[mask]
    count = len(df_filtered)
    return count
print(count_rows(df_testing))

def count_rows(df):
    mask1 = (df['gt_obj_id'] == 1)
    mask2 = (df['pred_obj_id'] == 0)
    mask3 = (df['ascaris_p_set'] == 0)
    mask4 = (df['trichuris_p_set'] == 0)
    mask5 = (df['hookworm_p_set'] == 0)
    mask6 = (df['schistosoma_p_set'] == 0)
    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
    df_filtered = df[mask]
    count = len(df_filtered)
    return count
print(count_rows(df_testing))

def count_rows(df):
    mask1 = (df['gt_obj_id'] == 2)
    mask2 = (df['pred_obj_id'] == 0)
    mask3 = (df['ascaris_p_set'] == 0)
    mask4 = (df['trichuris_p_set'] == 0)
    mask5 = (df['hookworm_p_set'] == 0)
    mask6 = (df['schistosoma_p_set'] == 0)
    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
    df_filtered = df[mask]
    count = len(df_filtered)
    return count
print(count_rows(df_testing))

def count_rows(df):
    mask1 = (df['gt_obj_id'] == 3)
    mask2 = (df['pred_obj_id'] == 0)
    mask3 = (df['ascaris_p_set'] == 0)
    mask4 = (df['trichuris_p_set'] == 0)
    mask5 = (df['hookworm_p_set'] == 0)
    mask6 = (df['schistosoma_p_set'] == 0)
    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
    df_filtered = df[mask]
    count = len(df_filtered)
    return count
print(count_rows(df_testing))

492
18
4
1


In [47]:
def count_rows(df, gt_obj_id):
    mask1 = (df['gt_obj_id'] == gt_obj_id)
    mask2 = (df['pred_obj_id'] == 0)
    mask3 = (df['ascaris_p_set'] == 0)
    mask4 = (df['trichuris_p_set'] == 0)
    mask5 = (df['hookworm_p_set'] == 0)
    mask6 = (df['schistosoma_p_set'] == 0)
    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
    df_filtered = df[mask]
    count = len(df_filtered)
    return count

print(count_rows(df_testing, '-'))
print(count_rows(df_testing, 1))
print(count_rows(df_testing, 2))
print(count_rows(df_testing, 3))


def count_rows(df, gt_obj_id):
    mask1 = (df['gt_obj_id'] == gt_obj_id)
    mask2 = (df['pred_obj_id'] == 1)
    mask3 = (df['ascaris_p_set'] == 0)
    mask4 = (df['trichuris_p_set'] == 0)
    mask5 = (df['hookworm_p_set'] == 0)
    mask6 = (df['schistosoma_p_set'] == 0)
    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
    df_filtered = df[mask]
    count = len(df_filtered)
    return count

print(count_rows(df_testing, '-'))
print(count_rows(df_testing, 0))
print(count_rows(df_testing, 2))
print(count_rows(df_testing, 3))


def count_rows(df, gt_obj_id):
    mask1 = (df['gt_obj_id'] == gt_obj_id)
    mask2 = (df['pred_obj_id'] == 2)
    mask3 = (df['ascaris_p_set'] == 0)
    mask4 = (df['trichuris_p_set'] == 0)
    mask5 = (df['hookworm_p_set'] == 0)
    mask6 = (df['schistosoma_p_set'] == 0)
    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
    df_filtered = df[mask]
    count = len(df_filtered)
    return count

print(count_rows(df_testing, '-'))
print(count_rows(df_testing, 0))
print(count_rows(df_testing, 1))
print(count_rows(df_testing, 3))

def count_rows(df, gt_obj_id):
    mask1 = (df['gt_obj_id'] == gt_obj_id)
    mask2 = (df['pred_obj_id'] == 3)
    mask3 = (df['ascaris_p_set'] == 0)
    mask4 = (df['trichuris_p_set'] == 0)
    mask5 = (df['hookworm_p_set'] == 0)
    mask6 = (df['schistosoma_p_set'] == 0)
    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
    df_filtered = df[mask]
    count = len(df_filtered)
    return count

print(count_rows(df_testing, '-'))
print(count_rows(df_testing, 0))
print(count_rows(df_testing, 1))
print(count_rows(df_testing, 2))

492
18
4
1
57
2
0
0
40
14
1
1
181
2
0
3


In [62]:
df_testing.to_csv('hinge_conf_score.csv',index=False)